In [5]:
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")

import numpy as np
import time

from read_scope_data import read_trc_data
from data_analysis_utils import Photons, PhotonPulse
from plot_utils import plot_counts_per_bin

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import fftconvolve

from plot_utils import plot_original_and_baseline, plot_subtracted_signal
%matplotlib qt

In [6]:
ifn = r"E:\x-ray\20241102\C3--E-ring-p30-z13-x200-xray--00011.trc"
xray_data, tarr_x = read_trc_data(ifn)

Reading data...
Done


In [11]:
time_ms = tarr_x * 1000
filtered_xray_data = - gaussian_filter1d(xray_data, sigma=10)

detector = Photons(time_ms,
                    filtered_xray_data,
                    threshold_multiplier=2,
                    filter_type='butterworth',
                    filter_value=0.000005)
detector.reduce_pulses()
pulse_times, pulse_areas = detector.get_pulse_arrays()


In [12]:
# Plot 0:
plt.figure(0)
plt.plot(time_ms, -xray_data)
plt.plot(time_ms, filtered_xray_data)

In [13]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_original_and_baseline(time_ms, -xray_data, detector, ax)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_subtracted_signal(time_ms, -xray_data, pulse_times, detector, ax)

In [5]:
# Create histogram of raw signal data
plt.figure(figsize=(10,6))
plt.hist(xray_data, bins=1000, density=True)
plt.xlabel('Signal Amplitude (V)') 
plt.ylabel('Density')
plt.title('Histogram of X-ray Signal Amplitudes')
plt.grid(True)
plt.show()

In [ ]:
# Create histogram of pulses over time with adjustable bin width
bin_width = 0.1

plt.figure(figsize=(10,6))
bins = np.arange(min(pulse_times), max(pulse_times) + bin_width, bin_width)
hist, bins = np.histogram(pulse_times, bins=bins)
bin_centers = (bins[:-1] + bins[1:])/2

plt.plot(bin_centers, hist)
plt.xlabel('Time (ms)')
plt.ylabel('Counts') 
plt.title(f'Photon Counts vs Time')
plt.grid(True)
plt.show()
